In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!mkdir /root/.kaggle
!(cp \
'/content/gdrive/My Drive/credentials/kaggle.json' \
'/root/.kaggle/')

# !kaggle competitions list
!(kaggle \
competitions \
list \
--category gettingStarted)

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                                              deadline             category            reward  teamCount  userHasEntered  
-------------------------------------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
https://www.kaggle.com/competitions/titanic                                      2030-01-01 00:00:00  Getting Started  Knowledge      16316            True  
https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques  2030-01-01 00:00:00  Getting Started  Knowledge       4519            True  
https://www.kaggle.com/competitions/spaceship-titanic                            2030-01-01 00:00:00  Getting Started  Knowledge       2639            True  
https://www.kaggle.com/competitions/digit-recognizer                             2030-01-01 00:00:00  Getting Started  Knowledge       1592           

In [ ]:
# !pwd
# !kaggle competitions download -c spaceship-titanic

In [ ]:
import os
import warnings

warnings.filterwarnings('ignore')
os.chdir('/content/gdrive/My Drive/RF/')
os.getcwd()

'/content/gdrive/My Drive/RF'

In [ ]:
import numpy as np
import pandas as pd
from sklearn import model_selection, ensemble, metrics

In [ ]:
main_df = pd.read_csv('raw_data/train.csv')

In [ ]:
for i in ['CryoSleep', 'VIP', 'Transported']:
    main_df[i] = main_df[i].astype(bool).astype(int)

del main_df['Name']
del main_df['Cabin']

In [ ]:
main_df = pd.concat([main_df,
                     pd.get_dummies(main_df['Destination'])],
                    axis=1)
del main_df['Destination']

In [ ]:
main_df = pd.concat([main_df,
                     pd.get_dummies(main_df['HomePlanet'])],
                    axis=1)
del main_df['HomePlanet']

In [ ]:
main_df = main_df.rename(columns={'55 Cancri e':'D1',
                                  'PSO J318.5-22':'D2',
                                  'TRAPPIST-1e':'D3'})

In [ ]:
uid = ['PassengerId']
features = ['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt',
            'ShoppingMall', 'Spa', 'VRDeck', 'D1', 'D2', 'D3',
            'Earth', 'Europa', 'Mars']
target = ['Transported']

In [ ]:
main_df = main_df[uid+features+target]

In [ ]:
main_df = main_df.fillna(0)

In [ ]:
main_df.head()

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,D1,D2,D3,Earth,Europa,Mars,Transported
0,0001_01,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,0,0
1,0002_01,0,24.0,0,109.0,9.0,25.0,549.0,44.0,0,0,1,1,0,0,1
2,0003_01,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,0,1,0,1,0,0
3,0003_02,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,0,1,0,1,0,0
4,0004_01,0,16.0,0,303.0,70.0,151.0,565.0,2.0,0,0,1,1,0,0,1


In [ ]:
train_df, test_df1 = model_selection.train_test_split(main_df, test_size=0.3)

In [ ]:
rf_clf = ensemble.RandomForestClassifier(n_estimators=500,
                                         max_depth=8,
                                         n_jobs=-1)
rf_clf = rf_clf.fit(train_df[features], train_df[target])

In [ ]:
train_df['Prediction'] = rf_clf.predict(train_df[features])
test_df1['Prediction'] = rf_clf.predict(test_df1[features])

In [ ]:
print(metrics.classification_report(train_df['Transported'], train_df['Prediction']))
print(metrics.classification_report(test_df1['Transported'], test_df1['Prediction']))

              precision    recall  f1-score   support

           0       0.85      0.80      0.82      3049
           1       0.81      0.85      0.83      3036

    accuracy                           0.83      6085
   macro avg       0.83      0.83      0.83      6085
weighted avg       0.83      0.83      0.83      6085

              precision    recall  f1-score   support

           0       0.80      0.79      0.79      1266
           1       0.80      0.82      0.81      1342

    accuracy                           0.80      2608
   macro avg       0.80      0.80      0.80      2608
weighted avg       0.80      0.80      0.80      2608



In [ ]:
test_df = pd.read_csv('raw_data/test.csv')

In [ ]:
for i in ['CryoSleep', 'VIP']:
    test_df[i] = test_df[i].astype(bool).astype(int)

del test_df['Name']
del test_df['Cabin']

In [ ]:
test_df = pd.concat([test_df,
                     pd.get_dummies(test_df['Destination'])],
                    axis=1)
del test_df['Destination']

test_df = pd.concat([test_df,
                     pd.get_dummies(test_df['HomePlanet'])],
                    axis=1)
del test_df['HomePlanet']

In [ ]:
test_df = test_df.rename(columns={'55 Cancri e':'D1',
                                  'PSO J318.5-22':'D2',
                                  'TRAPPIST-1e':'D3'})

In [ ]:
test_df = test_df[uid+features]

In [ ]:
test_df = test_df.fillna(0)

In [ ]:
test_df['Prediction'] = rf_clf.predict(test_df[features])

In [ ]:
submission_df = test_df[uid+['Prediction']]

In [ ]:
submission_df['Prediction'] = submission_df['Prediction'].astype(bool)
submission_df = submission_df.rename(columns={'Prediction':'Transported'})

In [ ]:
submission_df.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


In [ ]:
submission_df.to_csv('202204301639_rf_n500_d8.csv',
                     encoding='utf-8', index=False)

In [ ]:
# !(kaggle competitions submit \
# -c spaceship-titanic \
# -f 202204301639_rf_n500_d8.csv \
# -m "202204301639_rf_n500_d8")

100% 56.3k/56.3k [00:03<00:00, 18.1kB/s]
Successfully submitted to Spaceship Titanic